## Import libs

In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()
sess.close()

In [2]:
import os
import urllib
import numpy as np
import math

In [3]:
from pyspark.sql import *
from pyspark import SparkConf,SparkContext

from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType ,DoubleType ,BooleanType, IntegerType
from pyspark.sql.functions import udf ,col ,upper, avg

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor ,LinearRegression ,GeneralizedLinearRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.clustering import KMeans

from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor ,LinearRegression ,GeneralizedLinearRegression


from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.feature import PCA

from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler

In [4]:
sc=SparkContext()
sqlContext=SQLContext(sc)

In [5]:
#For gpu accelerated NN training:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Merge
from keras.regularizers import l2
from keras.optimizers import SGD

Using TensorFlow backend.


In [6]:
from sklearn.metrics import log_loss

In [7]:
print log_loss([0]*520  + [1]*480,[[.52,.48]]*1000)
print log_loss([0]*530  + [1]*470,[[.53,.47]]*1000)
print log_loss([0]*540  + [1]*460,[[.54,.46]]*1000)

print log_loss([0]*520  + [1]*480,[[.5,.5]]*1000)

0.69234696709
0.691346099002
0.689943758458
0.69314718056


In [10]:
np.version

<module 'numpy.version' from '/usr/local/lib/python2.7/dist-packages/numpy/version.pyc'>

# Check that nvida drivers are working:

In [9]:
!nvidia-smi

Mon Feb  6 12:09:29 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 860M    Off  | 0000:01:00.0     Off |                  N/A |
| N/A   49C    P0    N/A /  N/A |   3844MiB /  4043MiB |     14%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Read input

In [11]:
numerai_train_df     = sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai20170130/numerai_training_data.csv',inferSchema='true')
numerai_tournament_df= sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai20170130/numerai_tournament_data.csv',inferSchema='true')

print "length of training set:"
print (numerai_train_df.count())
print "length of tournament set:"
print (numerai_tournament_df.count())

length of training set:
136573
length of tournament set:
93498


In [12]:
#numerai_train_df.show(1,False)

## Convert input for use in ml lib algos

In [13]:
train_df=numerai_train_df.rdd.map(lambda x: (float(x.target),Vectors.dense([
                x.feature1
                ,x.feature2
                ,x.feature3
                ,x.feature4
                ,x.feature5
                ,x.feature6
                ,x.feature7
                ,x.feature8
                ,x.feature9
                ,x.feature10
                ,x.feature11
                ,x.feature12
                ,x.feature13
                ,x.feature14
                ,x.feature15
                ,x.feature16
                ,x.feature17
                ,x.feature18
                ,x.feature19
                ,x.feature20
                ,x.feature21
                ,x.feature22
                ,x.feature23
                ,x.feature24
                ,x.feature25
                ,x.feature26
                ,x.feature27
                ,x.feature28
                ,x.feature29
                ,x.feature30
                ,x.feature31
                ,x.feature32
                ,x.feature33
                ,x.feature34
                ,x.feature35
                ,x.feature36
                ,x.feature37
                ,x.feature38
                ,x.feature39
                ,x.feature40
                ,x.feature41
                ,x.feature42
                ,x.feature43
                ,x.feature44
                ,x.feature45
                ,x.feature46
                ,x.feature47
                ,x.feature48
                ,x.feature49
                ,x.feature50
                ]))).toDF(["label","features"])

tournament_df=numerai_tournament_df.rdd.map(lambda x: (x.t_id,Vectors.dense([
                x.feature1
                ,x.feature2
                ,x.feature3
                ,x.feature4
                ,x.feature5
                ,x.feature6
                ,x.feature7
                ,x.feature8
                ,x.feature9
                ,x.feature10
                ,x.feature11
                ,x.feature12
                ,x.feature13
                ,x.feature14
                ,x.feature15
                ,x.feature16
                ,x.feature17
                ,x.feature18
                ,x.feature19
                ,x.feature20
                ,x.feature21
                ,x.feature22
                ,x.feature23
                ,x.feature24
                ,x.feature25
                ,x.feature26
                ,x.feature27
                ,x.feature28
                ,x.feature29
                ,x.feature30
                ,x.feature31
                ,x.feature32
                ,x.feature33
                ,x.feature34
                ,x.feature35
                ,x.feature36
                ,x.feature37
                ,x.feature38
                ,x.feature39
                ,x.feature40
                ,x.feature41
                ,x.feature42
                ,x.feature43
                ,x.feature44
                ,x.feature45
                ,x.feature46
                ,x.feature47
                ,x.feature48
                ,x.feature49
                ,x.feature50
                ]))).toDF(["t_id","features"])



In [14]:
#(trainingData, testData) = train_df.randomSplit([0.6, 0.4],123456)
(trainingData, testData,everythingElse) = train_df.randomSplit([0.2, 0.1, 0.7],12345)

## Define log loss function

In [ ]:
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

## PCA

In [ ]:
# Do PCA and rename REPLACE the features column

pca = PCA(k=10, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(trainingData)
trainingDataPCA = model.transform(trainingData).drop("features")
trainingDataPCA = trainingDataPCA.withColumn("features",trainingDataPCA.pcaFeatures).drop("pcaFeatures")

testingDataPCA = model.transform(testData).drop("features")
testingDataPCA = testingDataPCA.withColumn("features",testingDataPCA.pcaFeatures).drop("pcaFeatures")

tournamentDataPCA = model.transform(tournament_df).drop("features")
tournamentDataPCA = tournamentDataPCA.withColumn("features",tournamentDataPCA.pcaFeatures).drop("pcaFeatures")

In [ ]:
#same thing WITHOUT renaming/replacing the features column
pca = PCA(k=10, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(trainingData)

trainingDataPCA = model.transform(trainingData)
testingDataPCA = model.transform(testData)
tournamentDataPCA = model.transform(tournament_df)

In [ ]:
#testingDataPCA.show(1,False)

In [ ]:
trainingData = trainingDataPCA
testData = testingDataPCA
tournament_df =tournamentDataPCA

## Clustering

In [ ]:
k=10
#k=5
#k=50
threshold1=0.69
threshold2=0.692

In [ ]:
kmeans = KMeans().setK(k).setSeed(1)
model = kmeans.fit(trainingData)
wssse = model.computeCost(trainingData)
print("Within Set Sum of Squared Errors = " + str(wssse))
centers = model.clusterCenters()

## Optimise for the number of clusters by finding the inflection point in cost vs number of clusters

In [ ]:
#for x in range(0, 100):
#    kmeans = KMeans().setK(x).setSeed(1)
#    model = kmeans.fit(trainingData)
#    wssse = model.computeCost(trainingData)
#    print("Within Set Sum of Squared Errors = " + str(wssse))
#    centers = model.clusterCenters()

In [ ]:
clustered_train_df = model.transform(trainingData)
clustered_train_df = clustered_train_df.withColumn("clusterNum",clustered_train_df.prediction).drop("prediction")
clustered_train_df.show(1)

clustered_test_df = model.transform(testData)
clustered_test_df = clustered_test_df.withColumn("clusterNum",clustered_test_df.prediction).drop("prediction")
clustered_test_df.show(1)

clustered_tournament_df = model.transform(tournament_df)
clustered_tournament_df = clustered_tournament_df.withColumn("clusterNum",clustered_tournament_df.prediction).drop("prediction")
clustered_tournament_df.show(1)


clustered_test_df.describe().show()

In [ ]:
#clustered_train_df.show(1,False)

# Neural Net with Keras

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
#only do this to group all columns containing vectors into a single vector column
assembler = VectorAssembler(
    inputCols=["features", "clusterNum","pcaFeatures"],
    outputCol="features2")
clustered_train_df2 = assembler.transform(clustered_train_df).drop('clusterNum').drop('features').drop("pcaFeatures")
clustered_test_df2  = assembler.transform(clustered_test_df).drop('clusterNum').drop('features').drop("pcaFeatures")

In [ ]:
assembler = VectorAssembler(
    inputCols=["features", "clusterNum"],
    outputCol="features2")
clustered_train_df2 = assembler.transform(clustered_train_df)
clustered_test_df2  = assembler.transform(clustered_test_df)

In [ ]:
#combine the different features columns with the k-means cluster output:

assembler1 = VectorAssembler(
    inputCols=["features", "clusterNum"],
    outputCol="features2")
clustered_train_df2 = assembler1.transform(clustered_train_df)
clustered_test_df2  = assembler1.transform(clustered_test_df)


assembler2 = VectorAssembler(
    inputCols=["pcaFeatures","clusterNum"],
    outputCol="pcafeatures2")
clustered_train_df2 = assembler2.transform(clustered_train_df2)
clustered_test_df2  = assembler2.transform(clustered_test_df2)


In [ ]:
clustered_test_df2.show(1)
clustered_test_df2.show(1)

In [ ]:
features    = np.array(clustered_train_df2.select("features2").toPandas())
pcaFeatures = np.array(clustered_train_df2.select("pcaFeatures2").toPandas())
labels      = np.array(clustered_train_df2.select("label").toPandas())

featuresT    = np.array(clustered_test_df2.select("features2").toPandas())
labelsT      = np.array(clustered_test_df2.select("label").toPandas())
pcaFeaturesT = np.array(clustered_test_df2.select("pcaFeatures2").toPandas())

In [ ]:
features    = np.array(clustered_train_df2.select("features2").toPandas())
labels      = np.array(clustered_train_df2.select("label").toPandas())

featuresT    = np.array(clustered_test_df2.select("features2").toPandas())
labelsT      = np.array(clustered_test_df2.select("label").toPandas())

In [ ]:
#features    = np.array(trainingData.select("features").toPandas())
#pcaFeatures = np.array(trainingData.select("pcaFeatures").toPandas())
#labels      = np.array(trainingData.select("label").toPandas())

#featuresT    = np.array(testData.select("features").toPandas())
#labelsT      = np.array(testData.select("label").toPandas())
#pcaFeaturesT = np.array(testData.select("pcaFeatures").toPandas())

In [ ]:
#print features

In [ ]:
features=clustered_train_df2.select("features2").toPandas()
labels  =clustered_train_df2.select("label").toPandas()

features = np.array(features)
labels =   np.array(labels)

featuresT =clustered_test_df2.select("features2").toPandas()
labelsT   =clustered_test_df2.select("label").toPandas()

featuresT = np.array(featuresT)
labelsT = np.array(labelsT)

In [ ]:
train_np     = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), features))[:,][:][:]
#train_np_pca = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), pcaFeatures))[:,][:][:]
train_res= np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), labels))[:,0]

In [ ]:
train_np_test     = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), featuresT))[:,0][:][:]
#train_np_pca_test = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), pcaFeaturesT))[:,0][:][:]
train_res_test    = np.array(map(lambda x: np.array(map(lambda y: np.array(y),x)), labelsT))[:,0]

In [ ]:
print type(train_np[0])
print train_np[10]

print train_res[10]
sum1= float(sum(train_res))
len1= float(len(train_res))
print "ratio: "+str(sum1/len1)

In [ ]:
print train_np.shape
train_np=train_np.reshape(train_np.shape[0],train_np.shape[2])
print train_np.shape

#print train_np_pca.shape
#train_np_pca=train_np_pca.reshape(train_np_pca.shape[0],train_np_pca.shape[2])
#print train_np_pca.shape

In [ ]:
model = Sequential()
model.add(Dense(11, input_dim=11, init='uniform', activation='relu'))
model.add(Dense(11, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, init='uniform', activation='relu'))
#model.add(Dense(11, init='uniform', activation='relu', W_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(5, init='uniform', activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, init='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
drop_out_rate=0.2

model = Sequential()

#model.add(Dropout(drop_out_rate, input_shape=(11,)))
model.add(Dense(20, input_dim=11, init='uniform', activation='relu'))

model.add(Dense(20, init='uniform', activation='relu'))
#model.add(Dropout(drop_out_rate))
model.add(Dense(400, init='uniform', activation='relu'))
model.add(Dropout(0.9))
#model.add(Dense(20, init='uniform', activation='relu'))
#model.add(Dropout(drop_out_rate))
#model.add(Dense(11, init='uniform', activation='relu', W_regularizer=l2(0.01)))
#model.add(Dropout(drop_out_rate))
#model.add(Dense(20, init='uniform', activation='relu'))
#model.add(Dropout(drop_out_rate))
#model.add(Dense(5, init='uniform', activation='relu'))

model.add(Dense(1, init='uniform', activation='sigmoid'))

#epochs = 200
#learning_rate = 1
#decay_rate = learning_rate / epochs
#momentum = 0.1
#sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
#model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_np, train_res, nb_epoch=epochs, batch_size=2000)

In [ ]:
#model.fit(train_np, train_res, nb_epoch=60, batch_size=1200)

evaluated_train = model.evaluate(train_np,train_res)
evaluated_test  = model.evaluate(train_np_test,train_res_test)

nn_answer       = model.predict(train_np_test)

print ""
print "---------------Train-------------------"
print "logloss : "+str(evaluated_train[0])
print "accuracy: "+str(evaluated_train[1]*100)+"%"
print "---------------Test--------------------"
print "logloss : "+str(evaluated_test[0])
print "accuracy: "+str(evaluated_test[1]*100)+"%"

In [ ]:
print nn_answer

# XGB

In [ ]:
import xgboost
from sklearn.metrics import accuracy_score

In [ ]:
model = xgboost.XGBClassifier()

#model.fit(train_np_pca, train_res)
model.fit(train_np, train_res)

# make predictions for test data
#y_pred = model.predict(train_np_pca_test)
y_pred = model.predict(train_np_test)

xgb_predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(train_res_test, xgb_predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
print type(np.array(xgb_predictions))
print type(nn_answer)

nn_answer=nn_answer.reshape(nn_answer.shape[0],)

print np.array(xgb_predictions)
print nn_answer

In [ ]:
trainingData.show()

train_xgb_udf = udf(lambda a: model.predict(a) , DoubleType())

trainingData2=trainingData.withColumn("xgboost_pred",train_xgb_udf(trainingData.features))

In [ ]:
train_keras_udf = udf(lambda a,b: final_model. , DoubleType())

In [ ]:
trainingData3=trainingData.withColumn("keras_pred",train_keras_udf(trainingData.features,trainingData.pcaFeatures))

# Bucket

In [ ]:
evaluated  = final_model.predict([train_np_test,train_np_pca_test])

In [ ]:
print evaluated.shape
print train_res_test.shape

In [ ]:
evaluated=evaluated.reshape(evaluated.shape[0],)

In [ ]:
evaluated

In [ ]:
train_res_test

In [ ]:
ensemble = evaluated

In [ ]:
#trainingData2.show()